# Viewing events from ROOT file

## Import modules and ROOT
Using ROOT 6 allows for interactive plots in notebook

In [1]:
from root_utils.root_file_utils import *
import numpy as np
import matplotlib.pyplot as plt
import math
from itertools import combinations
ROOT.enableJSVis()

Welcome to JupyROOT 6.18/00


## Load WCSim ROOT file

In [2]:
file = "/home/prouse/hk/WCSim-r6/gamma_100MeV.root"
wcsim = WCSimFile(file)
print(wcsim.nevent, "events in file")

number of entries in the geometry tree: 1
number of entries in the tree: 10
10 events in file


## Load event from file

In [3]:
ev = 2
wcsim.get_event(ev)
trigger = wcsim.get_first_trigger()
digi_hits = wcsim.get_digitized_hits()
true_hits = wcsim.get_true_hits()
photons = wcsim.get_hit_photons()
unique, counts = np.unique(true_hits["track"][true_hits["track"]>0], return_counts=True)
top4=unique[counts.argsort()[:-5:-1]].tolist()
print(top4, counts[counts.argsort()[:-5:-1]])

[3.0, 2.0, 25633.0, 25632.0] [265 167 107  70]


## Plot all photon emmision points in detector volume

In [4]:
cp = ROOT.TCanvas()
is_good_photon = photons["track"]>0
gphotons = ROOT.TGraph(len(photons["start_position"][is_good_photon,0]),
                       photons["start_position"][is_good_photon,0],
                       photons["start_position"][is_good_photon,2])
gphotons.SetMarkerStyle(20)
gphotons.SetMarkerSize(0.2)
gphotons.SetMarkerColor(ROOT.kBlack)
gphotons.SetTitle("")
gphotons.Draw("p")
cp.Draw()

## Plot all photon end points on photocathode (unrolled view)

In [5]:
cp = ROOT.TCanvas()
ghits = ROOT.TGraph(len(photons["end_position"][is_good_photon,0]),
                    np.arctan2(photons["end_position"][is_good_photon,2],
                               photons["end_position"][is_good_photon,0]),
                    photons["end_position"][is_good_photon,1])
ghits.SetMarkerStyle(20)
ghits.SetMarkerSize(0.2)
ghits.SetMarkerColor(ROOT.kBlack)
ghits.SetTitle("")
ghits.Draw("p")
cp.Draw()

## Plot all photon end points on photocathode (3D view)

In [6]:
cp = ROOT.TCanvas()
ghits2d = ROOT.TGraph2D("hits3d","",
                        len(photons["end_position"][is_good_photon,0]),
                        photons["end_position"][is_good_photon,0],
                        photons["end_position"][is_good_photon,2],
                        photons["end_position"][is_good_photon,1])
ghits2d.SetMarkerStyle(20)
ghits2d.SetMarkerSize(0.2)
ghits2d.SetMarkerColor(ROOT.kBlack)
ghits2d.SetTitle("")
ghits2d.Draw("p")
cp.Draw()

## Plot all hit PMTs coloured by hit charge (3D view)
Note: Due to a bug in ROOT javascript plotting, the colour scale does not usually work with interactive plotting

In [7]:
c = ROOT.TCanvas()
ncolors = ROOT.gStyle.GetNumberOfColors()
minq = min(digi_hits["charge"])
maxq = max(digi_hits["charge"])
qstep = (maxq-minq)/ncolors
gc = []
h = ROOT.TH3D("h",";x [cm]; z [cm]; y [cm]",10,-500,500,10,-500,500,10,-500,500)
h.SetMinimum(minq)
h.SetMaximum(maxq)
h.SetEntries(len(digi_hits["charge"]))
h.Draw()

for col in range(ncolors):
    sel = (digi_hits["charge"] >= (minq + qstep*col)) & (digi_hits["charge"] < (minq + qstep*(col+1)))
    if len(digi_hits["charge"][sel]) == 0:
        continue;
    #p = ROOT.TPolyMarker3D(len(digi_hits["charge"][sel]))
    #i = 0
    #for q, pos in zip(digi_hits["charge"][sel], digi_hits["position"][sel]):
    #    p.SetPoint(i, pos[0], pos[2], pos[1])
    #    i += 1
    #p.SetMarkerSize(1)
    #p.SetMarkerStyle(20)
    #p.SetMarkerColor(ROOT.TColor.GetColorPalette(col))
    #p.Draw()
    #gc.append(p)
    g = ROOT.TGraph2D("charge3d_"+str(col), "",
                      len(digi_hits["charge"][sel]),
                      digi_hits["position"][sel,0],
                      digi_hits["position"][sel,2],
                      digi_hits["position"][sel,1])
    g.SetMarkerStyle(20)
    g.SetMarkerSize(0.2)
    g.SetMarkerColor(ROOT.TColor.GetColorPalette(col))
    g.SetTitle("")
    g.Draw("psame")
    gc.append(g)
c.Draw()

## Plot all hit PMTs coloured by hit time (3D view)
Note: Due to a bug in ROOT javascript plotting, the colour scale does not usually work with interactive plotting

In [8]:
c = ROOT.TCanvas()
ncolors = ROOT.gStyle.GetNumberOfColors()
mint = 940
maxt = 1000
tstep = (maxt-mint)/ncolors
gc = []
h = ROOT.TH3D("h",";x [cm]; z [cm]; y [cm]",10,-500,500,10,-500,500,10,-500,500)
h.SetMinimum(mint)
h.SetMaximum(maxt)
h.SetEntries(len(digi_hits["time"]))
h.Draw()
for col in range(ncolors):
    sel = (digi_hits["time"] >= (mint + tstep*col)) & (digi_hits["time"] < (mint + tstep*(col+1)))
    if len(digi_hits["time"][sel]) == 0:
        continue;
    p = ROOT.TPolyMarker3D(len(digi_hits["time"][sel]))
    i = 0
    for q, pos in zip(digi_hits["time"][sel], digi_hits["position"][sel]):
        p.SetPoint(i, pos[0], pos[2], pos[1])
        i += 1
    p.SetMarkerSize(1)
    p.SetMarkerStyle(20)
    p.SetMarkerColor(ROOT.TColor.GetColorPalette(col))
    p.Draw()
    gc.append(p)
    #g = ROOT.TGraph2D("charge3d_"+str(col), "",
    #                  len(digi_hits["charge"][sel]),
    #                  digi_hits["position"][sel,0],
    #                  digi_hits["position"][sel,2],
    #                  digi_hits["position"][sel,1])
    #g.SetMarkerStyle(20)
    #g.SetMarkerSize(0.2)
    #tcol = ROOT.gROOT.GetColor(colt)
    #print(col, colt, tcol.GetRed(), tcol.GetGreen(), tcol.GetBlue())
    #g.SetMarkerColor(colt)
    #g.SetTitle("")
    #g.Draw("psame")
    #gc.append(g)
c.Draw()

Warning in <TFile::Append>: Replacing existing TH1: h (Potential memory leak).


## Plot WCSim tracks for electrons, muons, etc
The four tracks producing the most hits are coloured

In [9]:
c = ROOT.TCanvas()
def goodtrack(tr):
    if tr.GetFlag() == -1:
        return False
    ipnu = abs(tr.GetIpnu())
    KE = tr.GetE() - tr.GetM()
    if ipnu not in [11, 13, 15, 211, 2212]:
        return False
    if tr.GetE() < tr.GetM()/math.sqrt(1-(1/1.33)**2):
        return False
    return True
tracks = [tr for tr in trigger.GetTracks() if goodtrack(tr)]
xmin = min([tr.GetStart(0) for tr in tracks]+[tr.GetStop(0) for tr in tracks])
xmax = max([tr.GetStart(0) for tr in tracks]+[tr.GetStop(0) for tr in tracks])
zmin = min([tr.GetStart(2) for tr in tracks]+[tr.GetStop(2) for tr in tracks])
zmax = max([tr.GetStart(2) for tr in tracks]+[tr.GetStop(2) for tr in tracks])
f = c.DrawFrame(xmin-10, zmin-10, xmax+10, zmax+10)
f.GetXaxis().SetTitle("x-position [cm]")
f.GetYaxis().SetTitle("z-position [cm]")
lines = []
for tr in tracks:
    ipnu = abs(tr.GetIpnu())
    KE = tr.GetE() - tr.GetM()
    l = ROOT.TLine(tr.GetStart(0), tr.GetStart(2), tr.GetStop(0), tr.GetStop(2))
    if tr.GetId() in top4:
        col=2+top4.index(tr.GetId())
        colname = {2: "Red", 3: "Green", 4: "Blue", 5: "Yellow"}.get(col)
#        print(colname, tr.GetId(), ipnu, KE, tr.GetTime(), tr.GetStart(0), tr.GetStart(2), tr.GetStop(0), tr.GetStop(2))
        l.SetLineColor(col)
    lines.append(l)
    l.Draw()
c.Draw()

## Plot WCSim tracks electrons, muons, etc, with points showing emitted photons producing hits
The four tracks producing the most hits are coloured

In [10]:
c = ROOT.TCanvas()
def goodtrack(tr):
    if tr.GetFlag() == -1:
        return False
    ipnu = abs(tr.GetIpnu())
    KE = tr.GetE() - tr.GetM()
    if ipnu not in [11, 13, 15, 211, 2212]:
        return False
    if tr.GetE() < tr.GetM()/math.sqrt(1-(1/1.33)**2):
        return False
    return True
tracks = [tr for tr in trigger.GetTracks() if goodtrack(tr)]
xmin = min([tr.GetStart(0) for tr in tracks]+[tr.GetStop(0) for tr in tracks])
xmax = max([tr.GetStart(0) for tr in tracks]+[tr.GetStop(0) for tr in tracks])
zmin = min([tr.GetStart(2) for tr in tracks]+[tr.GetStop(2) for tr in tracks])
zmax = max([tr.GetStart(2) for tr in tracks]+[tr.GetStop(2) for tr in tracks])
f = c.DrawFrame(xmin-10, zmin-10, xmax+10, zmax+10)
f.GetXaxis().SetTitle("x-position [cm]")
f.GetYaxis().SetTitle("z-position [cm]")
graphs = []
for i in range(4):
    g = ROOT.TGraph(len(photons["start_position"][photons["track"] == top4[i],0]),
                    photons["start_position"][photons["track"] == top4[i],0],
                    photons["start_position"][photons["track"] == top4[i],2])
    g.SetMarkerStyle(20)
    g.SetMarkerSize(0.2)
    g.SetMarkerColor(2+i)
    g.SetTitle("")
    g.GetXaxis().SetTitle("x-position [cm]")
    g.GetYaxis().SetTitle("z-position [cm]")
    g.Draw("p")
    graphs.append(g)
g = ROOT.TGraph(len(photons["start_position"][~np.isin(photons["track"],top4),0]),
                photons["start_position"][~np.isin(photons["track"],top4),0],
                photons["start_position"][~np.isin(photons["track"],top4),2])
g.SetMarkerStyle(20)
g.SetMarkerSize(0.2)
g.SetMarkerColor(ROOT.kBlack)
g.Draw("p")
graphs.append(g)
lines = []
for tr in tracks:
    ipnu = abs(tr.GetIpnu())
    KE = tr.GetE() - tr.GetM()
    l = ROOT.TLine(tr.GetStart(0), tr.GetStart(2), tr.GetStop(0), tr.GetStop(2))
    if tr.GetId() in top4:
        col=2+top4.index(tr.GetId())
        colname = {2: "Red", 3: "Green", 4: "Blue", 5: "Yellow"}.get(col)
#        print(colname, tr.GetId(), ipnu, KE, tr.GetTime(), tr.GetStart(0), tr.GetStart(2), tr.GetStop(0), tr.GetStop(2))
        l.SetLineColor(col)
    lines.append(l)
    l.Draw()
c.Draw()
print(ROOT.kBlue)

600


## Plot hit PMTs (unrolled view)

In [11]:
c2 = ROOT.TCanvas()
is_barrel_digihit = np.abs(digi_hits["position"][:,1])<500
phis = np.arctan2(digi_hits["position"][is_barrel_digihit,2],digi_hits["position"][is_barrel_digihit,0])
ys = digi_hits["position"][is_barrel_digihit,1]
msize = 0.3
c2.DrawFrame(-math.pi, -550, math.pi, 550)
g1 = ROOT.TGraph(len(phis),phis,ys)
g1.SetMarkerStyle(20)
g1.SetMarkerSize(msize)
g1.SetMarkerColor(ROOT.kBlack)
g1.SetTitle("")
g1.Draw("p")
c2.Draw()

## Plot hit PMTs coloured by parent track
Hits that come solely due to one of the coloured tracks above have the corresponding colour
Hits from multiple tracks are coloured grey
Hits from other tracks are coloured black

In [12]:
c3 = ROOT.TCanvas()
is_barrel_hits = np.abs(true_hits["position"][:,1])<500
top4_hits = [is_barrel_hits & (true_hits["track"] == i) for i in top4]
many_hits = is_barrel_hits & (true_hits["track"] == -2)
other_hits = is_barrel_hits & ~np.isin(true_hits["track"],top4) & ~many_hits
phis = [np.arctan2(true_hits["position"][s,2], true_hits["position"][s,0]) for s in top4_hits]
phiother = np.arctan2(true_hits["position"][other_hits,2], true_hits["position"][other_hits,0])
phimany = np.arctan2(true_hits["position"][many_hits,2], true_hits["position"][many_hits,0])
ys = [true_hits["position"][s,1] for s in top4_hits]
yother = true_hits["position"][other_hits,1]
ymany = true_hits["position"][many_hits,1]
msize = 0.3
gs = []
c3.DrawFrame(-math.pi, -550, math.pi, 550)
for i in range(4):
    if len(phis[i]) == 0:
        continue
    g = ROOT.TGraph(len(phis[i]),np.array(phis[i]),np.array(ys[i]))
    g.SetMarkerStyle(20)
    g.SetMarkerSize(msize)
    g.SetMarkerColor(2+i)
    g.SetTitle("")
    g.Draw("p")
    gs.append(g)
if len(phiother) > 0:
    gother = ROOT.TGraph(len(phiother),np.array(phiother),np.array(yother))
    gother.SetMarkerStyle(20)
    gother.SetMarkerSize(msize)
    gother.SetMarkerColor(ROOT.kBlack)
    gother.Draw("psame")
if len(phimany) > 0:
    gmany = ROOT.TGraph(len(phimany),np.array(phimany),np.array(ymany))
    gmany.SetMarkerStyle(20)
    gmany.SetMarkerSize(msize)
    gmany.SetMarkerColor(ROOT.kGray)
    gmany.Draw("psame")
c3.Draw()
#print(len(phis[0]), len(phis[1]), len(phis[2]), len(phis[3]), len(phiother), len(phimany))

## Plot PMT hits due to individual tracks 

In [13]:
c = ROOT.TCanvas()
c.DrawFrame(-math.pi, -550, math.pi, 550)
gs[0].Draw("p")
c.Draw()

In [14]:
c = ROOT.TCanvas()
c.DrawFrame(-math.pi, -550, math.pi, 550)
gs[1].Draw("p")
c.Draw()

In [15]:
c = ROOT.TCanvas()
c.DrawFrame(-math.pi, -550, math.pi, 550)
gs[2].Draw("p")
c.Draw()

In [16]:
c = ROOT.TCanvas()
c.DrawFrame(-math.pi, -550, math.pi, 550)
gs[3].Draw("p")
c.Draw()

## Plot colour-coded hits in 3D together with true tracks in detector volume

In [17]:
is_barrel_hits = np.abs(true_hits["position"][:,1])<500
top4_hits = [is_barrel_hits & (true_hits["track"] == i) for i in top4]
many_hits = is_barrel_hits & (true_hits["track"] == -2)
other_hits = is_barrel_hits & ~np.isin(true_hits["track"],top4) & ~many_hits
xs = [true_hits["position"][s,0] for s in top4_hits]
xother = true_hits["position"][other_hits,0]
xmany = true_hits["position"][many_hits,0]
ys = [true_hits["position"][s,1] for s in top4_hits]
yother = true_hits["position"][other_hits,1]
ymany = true_hits["position"][many_hits,1]
zs = [true_hits["position"][s,2] for s in top4_hits]
zother = true_hits["position"][other_hits,2]
zmany = true_hits["position"][many_hits,2]
gs = []
c = ROOT.TCanvas()
c.SetGrid(0,0)
msize = 0.3
for i in range(4):
    if len(phis[i]) == 0:
        continue
    g = ROOT.TGraph2D("hits3d"+str(i),"",len(xs[i]),xs[i],zs[i],ys[i])
    g.SetMarkerStyle(20)
    g.SetMarkerSize(msize)
    g.SetMarkerColor(2+i)
    g.Draw("p" if i==0 else "psame")
    gs.append(g)
if len(phiother) > 0:
    gother = ROOT.TGraph2D("hits3dother","",len(xother),xother,zother,yother)
    gother.SetMarkerStyle(20)
    gother.SetMarkerSize(msize)
    gother.SetMarkerColor(ROOT.kBlack)
    gother.Draw("psame")
if len(phimany) > 0:
    gmany = ROOT.TGraph2D("hits3dmany","",len(xmany),xmany,zmany,ymany)
    gmany.SetMarkerStyle(20)
    gmany.SetMarkerSize(msize)
    gmany.SetMarkerColor(ROOT.kGray)
    gmany.Draw("psame")
lines = []
for tr in tracks:
    ipnu = abs(tr.GetIpnu())
    KE = tr.GetE() - tr.GetM()
    lx = np.array([tr.GetStart(0),tr.GetStop(0)])
    ly = np.array([tr.GetStart(1),tr.GetStop(1)])
    lz = np.array([tr.GetStart(2),tr.GetStop(2)])
    l = ROOT.TGraph2D("track"+str(tr.GetId()),"",2,lx,lz,ly)
    if tr.GetId() in top4:
        col=2+top4.index(tr.GetId())
        l.SetLineColor(col)
    lines.append(l)
    l.Draw("linesame")
c.Draw()